In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
# from pandas_datareader import data as pdr

/home/ash/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
from pandas_datareader import data as pdr

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pandas_datareader as pdr

# Function to fetch stock data and compute returns and covariance
def fetch_stock_data(stocks, start, end):
    try:
        data = pdr.get_data_yahoo(stocks, start, end)
        close_prices = data['Close']
        if close_prices.empty:
            raise ValueError("No data returned for the given stock symbols or date range.")
        daily_returns = close_prices.pct_change()
        avg_returns = daily_returns.mean()
        covariance_matrix = daily_returns.cov()
        return avg_returns, covariance_matrix
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return None, None

# List of stock tickers and date range
stock_symbols = ['CBA', 'BHP', 'TLS', 'NAB', 'WBC', 'STO']
tickers = [symbol + '.AX' for symbol in stock_symbols]
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=300)

# Get the mean returns and covariance matrix
mean_returns, cov_matrix = fetch_stock_data(tickers, start_date, end_date)

# Check if data retrieval was successful
if mean_returns is None or cov_matrix is None:
    print("Failed to retrieve stock data. Exiting.")
else:
    # Assign random portfolio weights
    portfolio_weights = np.random.random(len(mean_returns))
    portfolio_weights /= np.sum(portfolio_weights)  # Normalize to sum to 1

    # Monte Carlo parameters
    num_simulations = 400  # Number of simulations
    time_horizon = 100  # Number of days

    # Create a matrix to hold the mean returns repeated for each time step
    mean_matrix = np.full(shape=(time_horizon, len(portfolio_weights)), fill_value=mean_returns).T

    # Matrix to store simulated portfolio values
    portfolio_simulations = np.zeros((time_horizon, num_simulations))

    initial_investment = 10000  # Initial portfolio value

    # Perform Monte Carlo simulations
    for sim in range(num_simulations):
        random_values = np.random.normal(size=(time_horizon, len(portfolio_weights)))  # Uncorrelated random variables
        chol_matrix = np.linalg.cholesky(cov_matrix)  # Cholesky decomposition
        daily_returns_sim = mean_matrix + np.dot(chol_matrix, random_values.T)  # Correlated daily returns
        portfolio_simulations[:, sim] = np.cumprod(np.dot(portfolio_weights, daily_returns_sim) + 1) * initial_investment

    # Plot the simulation results
    plt.plot(portfolio_simulations)
    plt.ylabel('Portfolio Value ($)')
    plt.xlabel('Days')
    plt.title('Monte Carlo Simulation of Stock Portfolio')
    plt.show()

    # Value at Risk (VaR) calculation using Monte Carlo simulation results
    def calculate_mc_var(returns, alpha=5):
        """Calculates the VaR at a given confidence level (alpha)"""
        if isinstance(returns, pd.Series):
            return np.percentile(returns, alpha)
        else:
            raise TypeError("Expected a pandas Series.")

    # Conditional Value at Risk (CVaR) or Expected Shortfall
    def calculate_mc_cvar(returns, alpha=5):
        """Calculates the CVaR at a given confidence level (alpha)"""
        if isinstance(returns, pd.Series):
            var = calculate_mc_var(returns, alpha)
            return returns[returns <= var].mean()
        else:
            raise TypeError("Expected a pandas Series.")

    # Analyze portfolio simulation results
    portfolio_final_values = pd.Series(portfolio_simulations[-1, :])

    # Calculate VaR and CVaR
    var_5_percent = initial_investment - calculate_mc_var(portfolio_final_values, alpha=5)
    cvar_5_percent = initial_investment - calculate_mc_cvar(portfolio_final_values, alpha=5)

    # Display results
    print(f'VaR at 5% confidence: ${var_5_percent:.2f}')
    print(f'CVaR at 5% confidence: ${cvar_5_percent:.2f}')


Error fetching stock data: 'NoneType' object has no attribute 'group'
Failed to retrieve stock data. Exiting.


## Derivatives Parameter


In [4]:
import datetime

# Set initial parameters for the derivative
stock_price = 101.15   # Current price of the underlying stock (S)
strike_price = 98.01   # Option strike price (K)
volatility = 0.0991    # Volatility as a percentage (vol)
risk_free_rate = 0.01  # Risk-free interest rate (r)
num_time_steps = 10    # Number of time steps in the simulation (N)
num_simulations = 1000 # Number of Monte Carlo simulations (M)

option_market_price = 3.86 # Market price of the option

# Calculate time to maturity in years (T)
start_date = datetime.date(2022, 1, 17)
end_date = datetime.date(2022, 3, 17)
time_to_maturity = (end_date - start_date).days / 365

# Display the time to maturity
print(f"Time to Maturity (in years): {time_to_maturity}")


Time to Maturity (in years): 0.16164383561643836


## Vectorized Solution 

In [5]:
# Precompute constants used in the simulation
time_step = time_to_maturity / num_time_steps  # Size of each time step (dt)
drift = (risk_free_rate - 0.5 * volatility ** 2) * time_step  # Drift component (nudt)
volatility_sqrt_dt = volatility * np.sqrt(time_step)  # Volatility component scaled by time step (volsdt)
log_stock_price = np.log(stock_price)  # Logarithm of the initial stock price (lnS)

# Placeholders for summation used in Monte Carlo
sum_call_payoff = 0  # Sum of option payoffs (CT)
sum_call_payoff_squared = 0  # Sum of squared option payoffs (CT^2)

# Monte Carlo simulation to estimate option price
for sim in range(num_simulations):
    log_stock_price_t = log_stock_price
    # Simulate the stock price path over time steps
    for step in range(num_time_steps):
        log_stock_price_t += drift + volatility_sqrt_dt * np.random.normal()  # Random walk

    final_stock_price = np.exp(log_stock_price_t)  # Convert back to stock price (ST)
    call_payoff = max(0, final_stock_price - strike_price)  # Option payoff (CT)

    sum_call_payoff += call_payoff
    sum_call_payoff_squared += call_payoff ** 2

# Calculate the expected option price (C0) and standard error (SE)
option_price = np.exp(-risk_free_rate * time_to_maturity) * sum_call_payoff / num_simulations  # Discounted expectation
std_dev = np.sqrt((sum_call_payoff_squared - (sum_call_payoff ** 2) / num_simulations) * np.exp(-2 * risk_free_rate * time_to_maturity) / (num_simulations - 1))
standard_error = std_dev / np.sqrt(num_simulations)

# Print the results
print(f"Estimated Call Option Price: ${option_price:.2f}")
print(f"Standard Error: +/- ${standard_error:.2f}")


Estimated Call Option Price: $3.79
Standard Error: +/- $0.11
